<a href="https://colab.research.google.com/github/nhatduong01/SQL_on_BigQuery/blob/main/Advance/StackOverFlow_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
# Set up the Project Environment
import os
os.environ["GCLOUD_PROJECT"] = "BigQuerySQLExercises"

In [4]:
from google.cloud import bigquery
import pandas as pd


client = bigquery.Client()

data_ref = client.dataset(dataset_id= 'stackoverflow',
                          project = "bigquery-public-data")
dataset = client.get_dataset(data_ref)
tables = list(client.list_tables(dataset))
# See all the table ids in the table_list
for table in tables:
  print(table.table_id)

badges
comments
post_history
post_links
posts_answers
posts_moderator_nomination
posts_orphaned_tag_wiki
posts_privilege_wiki
posts_questions
posts_tag_wiki
posts_tag_wiki_excerpt
posts_wiki_placeholder
stackoverflow_posts
tags
users
votes


### 1) Time it took to receive an answer
We will write the query below, which focuses on questions asked in January 2018.  It returns a table with two columns:
- `q_id` - the ID of the question
- `time_to_answer` - how long it took (in seconds) for the question to receive an answer

In [10]:
query1 = """
         SELECT q.id AS questions_id,
         MIN (TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) AS time_to_answer
         FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
         LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
         ON q.id = a.parent_id
         WHERE q.creation_date >= '2018-01-01' AND q.creation_date < '2018-02-01'
         GROUP BY questions_id
         ORDER BY time_to_answer DESC"""
first_result = client.query(query1, project = 'bigquerysqlexercises').to_dataframe()
print("Percentage of answered questions: %s%%" % \
      (sum(first_result["time_to_answer"].notnull()) / len(first_result) * 100))
print("Number of questions:", len(first_result))
first_result.head()

Percentage of answered questions: 83.08578974324166%
Number of questions: 162059


,questions_id,time_to_answer
0,48209725,106537277.0
1,48207919,106399189.0
2,48173606,105876287.0
3,48350671,105875315.0
4,48275715,105768170.0


### 2) Inital question and Initial Question:
We will write a query that returns:
- `owner_user_id` - the user ID
- `q_creation_date` - the first time the user asked a question 
- `a_creation_date` - the first time the user contributed an answer 

In [17]:
query2 = """SELECT q.owner_user_id AS owner_user_id,
                   MIN(q.creation_date) AS q_creation_date,
                   MIN(a.creation_date) AS a_creation_date
                   FROM `bigquery-public-data.stackoverflow.posts_questions`AS q
                   FULL JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                   ON q.id = a.parent_id
                    WHERE q.creation_date >= '2019-01-01' AND q.creation_date < '2019-02-01' 
                    AND a.creation_date >= '2019-01-01' AND a.creation_date < '2019-02-01'
                   GROUP BY owner_user_id
                   """
query_job2 = client.query(query2, project = 'bigquerysqlexercises')
inial_actions = query_job2.to_dataframe()
inial_actions

,owner_user_id,q_creation_date,a_creation_date
0,9997832.0,2019-01-28 11:06:56.730000+00:00,2019-01-28 11:32:57.267000+00:00
1,9825326.0,2019-01-15 16:44:35.053000+00:00,2019-01-15 19:46:44.247000+00:00
2,5430776.0,2019-01-18 07:52:12.943000+00:00,2019-01-18 08:00:33.153000+00:00
3,83678.0,2019-01-28 18:28:57.800000+00:00,2019-01-28 18:38:02.247000+00:00
4,3545438.0,2019-01-10 09:36:11.570000+00:00,2019-01-10 09:43:53.880000+00:00
...,...,...,...
80672,6632820.0,2019-01-23 15:33:14.633000+00:00,2019-01-24 15:27:41.873000+00:00
80673,3769400.0,2019-01-21 16:25:16.237000+00:00,2019-01-21 17:11:59.320000+00:00
80674,10169709.0,2019-01-12 06:01:48.050000+00:00,2019-01-12 06:48:10.600000+00:00
80675,10404034.0,2019-01-12 00:19:07.810000+00:00,2019-01-12 00:35:37.770000+00:00


### 3) User in 2019:
We will write a query that returns the following columns:
- `id` - the IDs of all users who created Stack Overflow accounts in January 2019 (January 1, 2019, to January 31, 2019, inclusive)
- `q_creation_date` - the first time the user posted a question on the site; if the user has never posted a question, the value should be null
- `a_creation_date` - the first time the user posted a question on the site; if the user has never posted a question, the value should be null

In [23]:
query3 = """
         SELECT u.id AS id,
         MIN (q.creation_date) AS q_creation_date,
         MIN (a.creation_date) AS a_creation_date
         FROM `bigquery-public-data.stackoverflow.users` AS u
         LEFT JOIN `bigquery-public-data.stackoverflow.posts_questions` AS q
         ON u.id = q.owner_user_id
         LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
         on u.id = a.owner_user_id
         WHERE u.creation_date >= '2019-1-1' AND u.creation_date < '2019-2-1'
         GROUP BY id"""
job_query3 = client.query(query3, project = 'bigquerysqlexercises')

users_info = job_query3.to_dataframe()
users_info

,id,q_creation_date,a_creation_date
0,10920207,2019-03-05 09:59:13.160000+00:00,2019-12-16 07:35:23.797000+00:00
1,10876165,2019-01-19 18:11:50.987000+00:00,2021-04-09 17:36:47.330000+00:00
2,10937221,2021-01-07 15:18:33.160000+00:00,2019-01-19 12:43:23.087000+00:00
3,10926125,NaT,2019-04-11 06:33:24.787000+00:00
4,10882584,2019-12-27 11:15:17.197000+00:00,2020-01-02 10:40:49.437000+00:00
...,...,...,...
142007,10917433,NaT,2019-01-15 15:31:40.433000+00:00
142008,10929056,2020-01-14 11:14:53.627000+00:00,2019-01-17 16:06:45.893000+00:00
142009,10902853,2020-01-17 00:22:03.880000+00:00,NaT
142010,10916502,NaT,2019-01-15 10:53:53.643000+00:00


### 3) Distinct User posted on January 1, 2019:
In the code cell below,we will write a query that returns a table with a single column:
- `owner_user_id` - the IDs of all users who posted at least one question or answer on January 1, 2019.  Each user ID should appear at most once.

In [34]:
query4 = """
                  SELECT q.owner_user_id 
                  FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                  WHERE EXTRACT(DATE FROM q.creation_date) = '2019-01-01'
                  UNION ALL
                  SELECT a.owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_answers` AS a
                  WHERE EXTRACT(DATE FROM a.creation_date) = '2019-01-01'
         """
query_job4 = client.query(query4, project = 'bigquerysqlexercises')
distinct_users = query_job4.to_dataframe()
distinct_users

,owner_user_id
0,1621517.0
1,7587035.0
2,10849201.0
3,1711809.0
4,8773684.0
...,...
5683,3512189.0
5684,8949476.0
5685,8367626.0
5686,1491895.0
